# RNA-seq入门实战（一）：上游数据下载、格式转化和质控清洗

本节概览：
- 1.在文章中找到 GEO accession number, 从NCBI获取数据SRR号
- 2.在linux中使用prefetch命令根据SRR号下载SRA文件
- 3.使用fasterq-dump/fastq-dump命令将SRA文件转为FASTQ格式，pigz软件多线程压缩（可选）
- 4.使用fastqc和multiqc进行测序数据的质控查看
- 5.使用trim-galore去除低质量碱基和接头


## 一、从NCBI获取数据SRR号
数据的文章来源： [Formative pluripotent stem cells show features of epiblast cells poised for gastrulation | Cell Research (nature.com)](https://link.zhihu.com/?target=https%3A//www.nature.com/articles/s41422-021-00477-x%23data-availability)

在文章的Data availability 下找到 GEO accession number: `GSE154290`

进入NCBI官网搜索`GSE154290`，选择相应结果进入
<image src="../image/1.png">

找到**Supplementary file** 下的**SRA Run Select**选项

<image src="../image/2.png">

Common Fields下介绍了数据的基本信息，例如表中的**PAIRED**表示双端测序数据。此次实战选择勾选 Found 27 Items下的RNA_mESCs和RNA_EpiSCs各两个数据，再选中Select下的Selected选项，下载**Accession List**后复制数据的SRR号

<image src="../image/3.png">

<image src="../image/4.png">

## 二、SRA数据下载
1.创建并进入test项目文件夹，将SRR号粘贴导入idname文件

2.创建SRA数据下载的脚本文件

`vim 00_prefetch.sh`
主要利用了`sra-tools`中的 `prefetch`命令下载sra数据

```shell
#sh内容################################
echo -e "\n \n \n prefetch sra !!! \n \n \n "
date
mkdir -p /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/sra/
cd /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/sra/
pwd

cat  /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/idname | while read id ; \
do
      ( prefetch -O ./ $id & )
done
```


3.后台挂起运行脚本，运行情况导入log_00日志文件
`nohup bash 00_prefetch.sh >log_00 2>&1 &`
查看一下系统任务运行情况和test项目下的文件结构



查看一下系统任务运行情况和test项目下的文件结构

<image src="../image/5.png">

任务运行没问题，等待数据下载完毕，暂时去relax一下吧ヽ(￣▽￣)ﾉ 当cat log_00出现以下downloaded successfully字样时表示下载完成，再检查数据下载情况，确认下载完成没问题后就可以进行下一步文件格式转化啦

<image src="../image/6.png">

## 三、 SRA文件转为FASTQ格式
主要利用了sra-tool中的fasterq-dump命令转化格式为fastq，之后用pigz软件多线程压缩为.gz文件节省空间（可略过），再用fastqc和multiqc进行原始数据的质控和质控汇总~

格式转化: `fastq-dump` 或 `fasterq-dump` (速度更快,但没有--gzip选项)   
`--split-3 / -3 (fasterq) `会把原来双端拆分成两个文件,但是原来单端并不会保存成两个文件.        
`--gzip(fastq)` 输出gz格式, 能够节省空间的同时后续比对软件也都支持读取          
`-O `目录设置输出的目录              
`-e(fasterq)` 设置线程()          
`-p (fasterq)` 显示进度条       


同上，先创建 `01_sra2fq_qc1.sh` 脚本文件  
`vim 01_sra2fq_qc1.sh`
```shell

###########################################
#移动sra子文件夹下的文件并删除子文件夹
date
echo  -e "\n \n \n  111#  move files !!! \n \n \n  "
cd /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/sra
cat /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/idname | while read id
do
mv $id/*  ./
rm -rf $id/
done
date

echo  -e "\n \n \n  111#  sra>>>fq !!! \n \n \n  "
mkdir -p /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/fq/
cd /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/fq/
pwd
ls  /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/sra/*.sra |while read id 
do
echo " PROCESS $(basename $id) "
fasterq-dump -3 -e 12 -O ./ $id
pigz -p 12   /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/fq/*q
done
date

echo -e " \n \n \n  111# qc 1 !!! \n \n \n "       
mkdir /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/qc1/
cd  /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/qc1/
pwd
ls /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/fq/* | xargs fastqc -t 12 -o  ./
multiqc ./

echo -e  " \n 111#  ALL  Work Done!!! \n "
date

```

## 四、质控清洗
1. 原始数据质量查看

查看上一步qc1文件夹下的multiqc_report.html质控汇总网页文件，主要关注测序质量与测序接头这两项内容，可以发现该数据质量较好，平均质量均在30以上，接头含量也很低。 [具体内容分析见： 20160410 测序分析——使用 FastQC 做质控 - 知乎 (zhihu.com) 小L生信学习日记-3丨原始数据质量如何判断？-上 (qq.com) 小L生信学习日记-4丨原始数据质量如何判断？-下 - 知乎 (zhihu.com)](https://zhuanlan.zhihu.com/p/20731723)


### FastQC安装

#### 安装包安装
```shell
wget命令下载包（官网）
#download fastqc paskage
wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.12.1.zip
# 配置fastqc环境，在~/.bashrc文件（或者在其他文件配置）末尾加入以下代码：
#dispose enviroment
# fastQC
#格式如下：
#export PATH="包的路径：$PATH"
export PATH="/home/cyh/biosoft/FastQC: $PATH" #路径不一样，请勿完全抄，仅供参考

```

### multiqc

`conda install -c bioconda multiqc`

#### conda 安装
```shell
conda search bioconda fastqc            #查询conda中是否有fastqc安装包，并返回所有版本
conda install -c bioconda fastqc                   #默认安装最新版fastqc
```

### 基本格式

```text
# fastqc [-o output dir] [--(no)extract] [-f fastq|bam|sam] [-c contaminant file] seqfile1 .. seqfileN

# 主要是包括前面的各种选项和最后面的可以加入N个文件
# -o --outdir FastQC生成的报告文件的储存路径，生成的报告的文件名是根据输入来定的
# --extract 生成的报告默认会打包成1个压缩文件，使用这个参数是让程序不打包
# -t --threads 选择程序运行的线程数，每个线程会占用250MB内存，越多越快咯
# -c --contaminants 污染物选项，输入的是一个文件，格式是Name [Tab] Sequence，里面是可能的污染序列，如果有这个选项，FastQC会在计算时候评估污染的情况，并在统计的时候进行分析，一般用不到
# -a --adapters 也是输入一个文件，文件的格式Name [Tab] Sequence，储存的是测序的adpater序列信息，如果不输入，目前版本的FastQC就按照通用引物来评估序列时候有adapter的残留
# -q --quiet 安静运行模式，一般不选这个选项的时候，程序会实时报告运行的状况。
```

以我平时用的一个真实的例子：

```shell
fastqc -o ./tmp.result/fastQC/ -t 6 ./tmp.data/fastq/H1EScell-dnase-2014-GSE56869_20151208_SRR1248176_1.fastq 
```

#### 使用浏览器打开后缀是html的文件，就是图表化的fastqc报告：
<image src="../image/7.webp">

**FastQC的报告介绍**

- 总结信息


上图中Summary的部分就是整个报告的目录，整个报告分成若干个部分。合格会有个绿色的对勾，警告是个“!”，不合格是个红色的叉子。

- 基本信息

<image src="../image/8.webp">

```text
# Encoding指测序平台的版本和相应的编码版本号，这个在计算Phred反推error P的时候有用，如果不明白可以参考之前的文章。
# Total Sequences记录了输入文本的reads的数量
# Sequence length 是测序的长度
# %GC 是我们需要重点关注的一个指标，这个值表示的是整体序列中的GC含量，这个数值一般是物种特意的，比如人类细胞就是42%左右。
```

- 序列测序质量统计
<image src="../image/9.webp">

```text
# 此图中的横轴是测序序列第1个碱基到第101个碱基
# 纵轴是质量得分，Q = -10*log10（error P）即20表示1%的错误率，30表示0.1%
# 图中每1个boxplot，都是该位置的所有序列的测序质量的一个统计，上面的bar是90%分位数，下面的bar是10%分位数，箱子的中间的横线是50%分位数，箱子的上边是75%分位数，下边是25%分位数
# 图中蓝色的细线是各个位置的平均值的连线
# 一般要求此图中，所有位置的10%分位数大于20,也就是我们常说的Q20过滤
# 所以上面的这个测序结果，需要把后面的87bp以后的序列切除，从而保证后续分析的正确性
# Warning 报警 如果任何碱基质量低于10,或者是任何中位数低于25
# Failure 报错 如果任何碱基质量低于5,或者是任何中位数低于20
```

- 每个tail测序的情况
<image src="../image/10.webp">

```text
# 横轴和之前一样，代表101个碱基的每个不同位置
# 纵轴是tail的Index编号
# 这个图主要是为了防止，在测序过程中，某些tail受到不可控因素的影响而出现测序质量偏低
# 蓝色代表测序质量很高，暖色代表测序质量不高，如果某些tail出现暖色，可以在后续分析中把该tail测序的结果全部都去除
```
- 每条序列的测序质量统计
<image src="../image/11.webp">

```text
# 假如我测的1条序列长度为101bp，那么这101个位置每个位置Q之的平均值就是这条reads的质量值
# 该图横轴是0-40，表示Q值
# 纵轴是每个值对应的reads数目
# 我们的数据中，测序结果主要集中在高分中，证明测序质量良好！
```
- GC 含量统计
<image src="../image/12.webp">
```shell
# 横轴是1 - 101 bp；纵轴是百分比
# 图中四条线代表A T C G在每个位置平均含量
# 理论上来说，A和T应该相等，G和C应该相等，但是一般测序的时候，刚开始测序仪状态不稳定，很可能出现上图的情况。像这种情况，即使测序的得分很高，也需要cut开始部分的序列信息，一般像我碰到这种情况，会cut前面5bp
```

- 序列平均GC含量分布图
<image src="../image/13.webp">
```text
# 横轴是0 - 100%； 纵轴是每条序列GC含量对应的数量
# 蓝色的线是程序根据经验分布给出的理论值，红色是真实值，两个应该比较接近才比较好
# 当红色的线出现双峰，基本肯定是混入了其他物种的DNA序列
# 这张图中的信息良好
```

- 序列测序长度统计
<image src="../image/14.webp">

```text
# 每次测序仪测出来的长度在理论上应该是完全相等的，但是总会有一些偏差
# 比如此图中，101bp是主要的，但是还是有少量的100和102bp的长度，不过数量比较少，不影响后续分析
# 当测序的长度不同时，如果很严重，则表明测序仪在此次测序过程中产生的数据不可信 

```

- 序列Adapter

<image src="../image/15.webp">


```text
# 此图衡量的是序列中两端adapter的情况
# 如果在当时fastqc分析的时候-a选项没有内容，则默认使用图例中的四种通用adapter序列进行统计
# 本例中adapter都已经去除，如果有adapter序列没有去除干净的情况，在后续分析的时候需要先使用cutadapt软件进行去接头，这个软件以后我会介绍
```


- 重复短序列

<image src="../image/16.webp">


```text
# 这个图统计的是，在序列中某些特征的短序列重复出现的次数
# 我们可以看到1-8bp的时候图例中的几种短序列都出现了非常多的次数，一般来说，出现这种情况，要么是adapter没有去除干净，而又没有使用-a参数；要么就是序列本身可能重复度比较高，如建库PCR的时候出现了bias
# 对于这种情况，我的办法是可以cut掉前面的一些长度，可以试着cut 5~8bp

```

## 2. 质控清洗数据
主要使用`trim-galore`去除低质量碱基和接头，详尽使用方法参见[lncRNA组装流程的软件介绍之`trim-galore`](https://link.zhihu.com/?target=https%3A//mp.weixin.qq.com/s%3Fsearch_click_id%3D2253394900344533860-1633341597788-628761%26sub%3D%26__biz%3DMzAxMDkxODM1Ng%3D%3D%26mid%3D2247503527%26idx%3D4%26sn%3D261be2c7ff2a7f80b2e6ab139028d780%26chksm%3D9b4b8e1cac3c070a963f59fcc55095fe0ae37d1eecde3e67f6682779c91d0e907a8fd7eb8f84%26scene%3D3%26subscene%3D10000%26clicktime%3D1633341597%26enterid%3D1633341597%26ascene%3D0%26devicetype%3Dandroid-30%26version%3D28000f35%26nettype%3Dcmnet%26abtest_cookie%3DAAACAA%253D%253D%26lang%3Dzh_CN%26exportkey%3DAdXQxZkOzbrJK%252BUwprsoMEk%253D%26pass_ticket%3DpvvLsQPcUT6hjrE3KSHbHTaZKL%252FVtNBVq%252BLbY9r0hucz%252FDdbU3NgO9ofB9mtC3fS%26wx_header%3D1) 常用参数如下：

<image src="../image/17.png" title="trim-galore常用参数">

### 安装

```shell
conda install trim-galore
# 或者源码安装

curl -fsSL https://github.com/FelixKrueger/TrimGalore/archive/0.4.5.tar.gz -o trim_galore.tar.gz 

tar xvzf trim_galore.tar.gz

TrimGalore-0.4.5/trim_galore            #检查是否安装成功

```
安装cutadapt，遇到python报错就更新python版本

```shell
conda install -c bioconda cutadapt -y
```

### 参数
```shell
--quality: 设定Phred quality score间值, 默认为 200 分析时可改成 25 , 稍微严格一些。
--phred33：：选择-phred33或者-phred64，表示测序平台使用的Phred quality score。具体怎么选择，看你用什么则序平台；例如：-phred33对应（Sanger/Illumina 1.9+ encoding），-phred64对应（Illumina 1.5 encoding)
--adapter：输入adapter序列。她可以不输入，Trim Galore!会自动寻找可能性最高的平台对应的adapter。自动㩭选的平台三个，也直接显式输入这三种平台，即--illumina、--nextera和--small_rna。
--stringency: 设定可以忍受的前后adapter重查的碱基数，點认为 1 （非常苛刻）。可以适度放宽，因为后一个 adapter几乎不可能被则序仪读到。
--length: 设定输出reads长度果值，小于设定值会被拋弃。
--paired: 对于双媏则序结果, 一对reads中, 如果有一个被剔除, 那么另一个会被同样抛弃, 而不管是歪达到标准。
--retain_unpaired: 对于双端则序结果，一对reads中，如果一个readj达到标准，但是对应的另一个要被抛弃，达到标准的read会被单独保存为一个文件。
--gzip和--dont_gzip: 清洗后的数据zip打包或者不打包。
--output_dir: 输入目录。需要提前建立目录, 否则运行会报错。
--trim-n : 移除read-端的方reads
```


### 脚本
`vim 2_cleanfq_qc2.sh`
```shell

##############################################
echo -e " \n \n \n 222# Clean ! trim_galore !!! \n \n \n"
date
mkdir /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/
cd /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/
pwd

##########single###########################################################################
#ls ~/test/raw/fq/*.f* | while read id 
#do 
#       trim_galore -q 25 -j 4 --phred33 --length 35 --stringency 3 \
#               --gzip  -o ~/test/clean/  $id 
#done
#
##########paired###########################################################################
#1）先把文件_1、_2的路径文件名分别存储，再合并成两列的格式，存为config#########
  ls /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/fq/*_1*  >1
  ls /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/fq/*_2*  >2
  paste 1 2 >config
  cat config | while read id
  do
    arr=($id)
    fq1=${arr[0]}
    fq2=${arr[1]}
   trim_galore  -j 4  -q 25  --phred33 --length 35 --stringency 3 \
               --paired --gzip -o /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/  $fq1 $fq2
  done
###########################################################################################                  


echo -e "\n \n \n 222# qc2 检查clean清洗结果!!! \n \n \n"
mkdir  /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/qc2
cd /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/qc2
pwd
ls /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/*f*.gz | xargs fastqc -t 12 -o   /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/qc2
multiqc   ./

echo -e " \n 222# ALL  Work Done !!! \n "
date

```

`nohup bash  2_cleanfq_qc2.sh >log_2 2>&1 &`

### 清洗后数据质量查看
查看~/test/clean/qc2下的multiqc_report.html质控汇总网页文件，碱基质量更好一些了

<image src="../image/18.png" title="">

> 到此，我们完成了RNAseq原始数据的下载、格式转化和质控清洗步骤，得到了经过质控后存放于clean文件夹下的fastq文件，接下来就可以利用这些cleaned fastq文件进行下一步的比对、计数（hisat2+feature_counts 或 salmon），最终得到我们想要的counts文件